In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import io

In [ ]:
import pandas as pd

# Load Excel

path = "/content/gdrive/MyDrive/Data insight 2025/Star_Rating.xlsx"
excel_file = pd.ExcelFile(path)
print(excel_file.sheet_names)
df= excel_file.parse("Star Rating 2024")

# Check first 5 rows & columns info
print(df.head())
print(df.info())
print(df['SNP'].unique())
print(df['2024 Overall'].unique())


['Star Rating 2021', 'Star Rating 2022', 'Star Rating 2023', 'Star Rating 2024']
  Contract Number   SNP                 2024 Overall  Enrollment
0           E3014   No                Not Applicable       95075
1           H0022   No     Not enough data available       12019
2           H1277  Yes     Not enough data available          17
3           H6316  Yes     Not enough data available          24
4           H6378  Yes   Plan too new to be measured          24
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Contract Number  786 non-null    object
 1   SNP              786 non-null    object
 2   2024 Overall     786 non-null    object
 3   Enrollment       786 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 24.7+ KB
None
['No ' 'Yes ']
['Not Applicable' 'Not enough data available'
 'Plan too new to be measured' '2' '4'

In [ ]:
df['Overall'] = pd.to_numeric(df['2024 Overall'], errors='coerce')

# Check how many NaNs we got
print(f"Number of NaNs in Overall: {df['Overall'].isna().sum()}")


Number of NaNs in Overall: 242


In [ ]:
df_clean = df.dropna(subset=['Overall']).copy()

print(f"Clean dataset size: {df_clean.shape}")
print(df_clean['SNP'].unique())
print(df_clean['Overall'].describe())


Clean dataset size: (544, 5)
['No ' 'Yes ']
count    544.000000
mean       3.625000
std        0.682591
min        2.000000
25%        3.000000
50%        3.500000
75%        4.000000
max        5.000000
Name: Overall, dtype: float64


In [ ]:
df_clean['SNP'] = df_clean['SNP'].str.strip().str.capitalize()

print(df_clean['SNP'].unique())


['No' 'Yes']


In [ ]:
group_yes = df_clean[df_clean['SNP'] == 'Yes']['Overall']
group_no = df_clean[df_clean['SNP'] == 'No']['Overall']

print(f"Group Yes size: {len(group_yes)}")
print(f"Group No size: {len(group_no)}")


Group Yes size: 302
Group No size: 242


In [ ]:
from scipy.stats import ttest_ind

t_stat, p_val = ttest_ind(group_yes, group_no, equal_var=False)
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_val:.4f}")


T-statistic: -1.1752
P-value: 0.2404


To analyze whether there is a significant difference in the Star Ratings between contracts that offer Special Needs Plans (SNP) and those that do not, I conducted an independent two-sample t-test on the 2024 Star Rating data. Before analysis, the “2024 Overall” Star Ratings were converted to numeric values, with non-numeric entries treated as missing and removed from the dataset.

The t-test results were:

T-statistic = -1.1752

P-value = 0.2404

Since the p-value is greater than the common significance threshold of 0.05, we fail to reject the null hypothesis. This means there is no statistically significant difference in the average Star Ratings between SNP and non-SNP contracts for the year 2024. In other words, being a special needs plan does not appear to affect the quality rating assigned to the contract based on this dataset. *italicised text*


In [ ]:
df_2021 = excel_file.parse("Star Rating 2021")
df_2022 = excel_file.parse("Star Rating 2022")
df_2023 = excel_file.parse("Star Rating 2023")
df_2024 = excel_file.parse("Star Rating 2024")

In [ ]:
# Merge them on 'Contract Number'
df = df_2021.merge(df_2022, on='Contract Number', how='outer', suffixes=('_2021', '_2022'))
df = df.merge(df_2023, on='Contract Number', how='outer')
df = df.merge(df_2024, on='Contract Number', how='outer')

df.rename(columns={
     '2021 Overall': 'Overall_2021',
    '2022 Overall': 'Overall_2022',
    '2023 Overall': 'Overall_2023',
    '2024 Overall': 'Overall_2024'
}, inplace=True)

In [ ]:
df.head()

,Contract Number,Overall_2021,Overall_2022,Overall_2023,SNP,Overall_2024,Enrollment
0,E0654,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN
1,E3014,NaN,NaN,NaN,No,Not Applicable,95075.0
2,E3014,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN
3,E4744,Not Applicable,Not Applicable,Not Applicable,NaN,NaN,NaN
4,H0022,NaN,NaN,NaN,No,Not enough data available,12019.0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
star_2021 = excel_file.parse("Star Rating 2021")
star_2022 = excel_file.parse("Star Rating 2022")
star_2023 = excel_file.parse("Star Rating 2023")
star_2024 = excel_file.parse("Star Rating 2024")

In [ ]:
def clean_star(df, overall_col):
    df[overall_col] = df[overall_col].replace(
        ['Not Applicable', 'Not enough data available', 'Plan too new to be measured'],
        np.nan
    ).astype(float)
    return df


In [ ]:
star_2021 = clean_star(star_2021, '2021 Overall')
star_2022 = clean_star(star_2022, '2022 Overall')
star_2023 = clean_star(star_2023, '2023 Overall')
star_2024 = clean_star(star_2024, '2024 Overall')

In [ ]:
star_2021.head()

,Contract Number,2021 Overall
0,E0654,NaN
1,E3014,NaN
2,E4744,NaN
3,H0022,NaN
4,H0028,4.0


In [ ]:
merged = star_2021[['Contract Number', '2021 Overall']].merge(
    star_2022[['Contract Number', '2022 Overall']], on='Contract Number', how='outer')

merged = merged.merge(
    star_2023[['Contract Number', '2023 Overall']], on='Contract Number', how='outer')

merged = merged.merge(
    star_2024[['Contract Number', '2024 Overall']], on='Contract Number', how='outer')


In [ ]:
merged = merged.merge(
    star_2024[['Contract Number', 'SNP', 'Enrollment']], on='Contract Number', how='left')


In [ ]:
merged.head()

,Contract Number,2021 Overall,2022 Overall,2023 Overall,2024 Overall,SNP,Enrollment
0,E0654,NaN,NaN,NaN,NaN,NaN,NaN
1,E3014,NaN,NaN,NaN,NaN,No,95075.0
2,E3014,NaN,NaN,NaN,NaN,NaN,NaN
3,E4744,NaN,NaN,NaN,NaN,NaN,NaN
4,H0022,NaN,NaN,NaN,NaN,No,12019.0


In [ ]:
# List of columns to clean
overall_cols = ['2021 Overall', '2022 Overall', '2023 Overall', '2024 Overall']

# Values to replace with 1
replace_vals = [
    "Not Applicable",
    "Not enough data available",
    "Plan too new to be measured"
]

# Replace strings with 1
merged[overall_cols] = merged[overall_cols].replace(replace_vals, 1)

# Replace NaNs with 0
merged[overall_cols] = merged[overall_cols].fillna(0)

# Optionally convert all values to float (recommended if you'll do math/modeling)
merged[overall_cols] = merged[overall_cols].astype(float)


In [ ]:
merged.head()

,Contract Number,2021 Overall,2022 Overall,2023 Overall,2024 Overall,SNP,Enrollment
0,E0654,0.0,0.0,0.0,0.0,NaN,NaN
1,E3014,0.0,0.0,0.0,0.0,No,95075.0
2,E3014,0.0,0.0,0.0,0.0,NaN,NaN
3,E4744,0.0,0.0,0.0,0.0,NaN,NaN
4,H0022,0.0,0.0,0.0,0.0,No,12019.0


In [ ]:
model_data = merged.dropna(subset=['2024 Overall']).reset_index(drop=True)


In [ ]:
X = model_data[['2021 Overall', '2022 Overall', '2023 Overall']].copy()
# === STEP 5: Fill missing feature values (NaNs) with column-wise mean ===
X = X.apply(pd.to_numeric, errors='coerce')  # Ensure values are numeric
X = X.fillna(X.mean())
y = model_data['2024 Overall']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# === STEP 9: Evaluate the model ===
y_pred = model.predict(X_test)

print("R^2 Score:", r2_score(y_test, y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

R^2 Score: 0.10784564894948223
Mean Squared Error: 2.380396934655309


In [ ]:

# === STEP 10: Predict 2025 ratings using all available data ===
# We'll use latest known ratings (2021–2023) to predict 2025
future_data = merged[['2021 Overall', '2022 Overall', '2023 Overall']].copy()
future_data = future_data.apply(pd.to_numeric, errors='coerce')
future_data = future_data.fillna(future_data.mean())

merged['Predicted_2025'] = model.predict(future_data)

# === STEP 11: Show predicted results ===
print(merged[['Contract Number', '2021 Overall', '2022 Overall', '2023 Overall', 'Predicted_2025']])

     Contract Number  2021 Overall  2022 Overall  2023 Overall  Predicted_2025
0             E0654            0.0           0.0           0.0        1.652222
1              E3014           0.0           0.0           0.0        1.652222
2             E3014            0.0           0.0           0.0        1.652222
3             E4744            0.0           0.0           0.0        1.652222
4              H0022           0.0           0.0           0.0        1.652222
...              ...           ...           ...           ...             ...
1731          S9325            0.0           0.0           0.0        1.652222
1732          S9409            0.0           0.0           0.0        1.652222
1733           S9701           0.0           0.0           0.0        1.652222
1734          S9701            0.0           0.0           0.0        1.652222
1735          S9854            0.0           0.0           0.0        1.652222

[1736 rows x 5 columns]
